In [1]:
import pandas as pd

ACADEMIC_PATH = r'../../data/raw/academic_records.csv'
ADMISSION_PATH = r'../../data/raw/admission.csv'
TEST_PATH = r'../../data/raw/test.csv'
academic_records = pd.read_csv(ACADEMIC_PATH)
admission = pd.read_csv(ADMISSION_PATH)

In [2]:
import os
os.environ['PYTHONHASHSEED'] = '42'

import random
import numpy as np

random.seed(42)
np.random.seed(42)
np.random.set_state(np.random.RandomState(42).get_state())

In [3]:
import pandas as pd
import numpy as np
import re

# --- HÀM MỚI: Xử lý định dạng học kỳ phức tạp ---
def parse_semester_string(sem_str):
    """
    Chuyển đổi chuỗi như 'HK1 2023-2024' thành mã số 20231 để sort được.
    Logic: Năm * 10 + Kỳ
    """
    s = str(sem_str).strip()
    
    # Trường hợp 1: Dạng số sẵn (VD: 20231)
    if s.isdigit():
        return int(s)
    
    # Trường hợp 2: Dạng chữ (VD: HK1 2023-2024 hoặc Học kỳ 1 năm 2023)
    # Tìm tất cả các con số trong chuỗi
    digits = re.findall(r'\d+', s)
    
    if len(digits) >= 2:
        # Giả sử số nhỏ là kỳ, số lớn (4 chữ số) là năm
        # Tìm năm (thường là số có 4 chữ số đầu tiên tìm thấy)
        years = [int(d) for d in digits if len(d) == 4]
        sems = [int(d) for d in digits if len(d) == 1]
        
        if years and sems:
            year = years[0]
            sem = sems[0]
            return year * 10 + sem
            
    return 0 # Không xác định

def clean_data_pipeline_v3(admission, academic_records):
    print("--- 🚀 BẮT ĐẦU QUY TRÌNH LÀM SẠCH DỮ LIỆU (FIXED VERSION) ---")
    
    adm = admission.copy()
    acad = academic_records.copy()
    
    # 1. Chuẩn hóa ID
    adm['MA_SO_SV'] = adm['MA_SO_SV'].astype(str)
    acad['MA_SO_SV'] = acad['MA_SO_SV'].astype(str)
    
    # ---------------------------------------------------------
    # [FIX QUAN TRỌNG] Xử lý HOC_KY thông minh hơn
    # ---------------------------------------------------------
    print("-> Đang xử lý cột HOC_KY...")
    # Tạo cột HOC_KY_INT dùng để sort (VD: 20231)
    acad['HOC_KY_INT'] = acad['HOC_KY'].apply(parse_semester_string)
    
    # Kiểm tra xem có dòng nào bị lỗi (bằng 0) không
    error_count = (acad['HOC_KY_INT'] == 0).sum()
    if error_count > 0:
        print(f"   ⚠️ Cảnh báo: Có {error_count} dòng không đọc được HOC_KY.")

    # Merge dữ liệu
    df = pd.merge(acad, adm, on='MA_SO_SV', how='left')
    
    # Sắp xếp theo Time-series chuẩn xác dựa trên cột vừa tạo
    df = df.sort_values(by=['MA_SO_SV', 'HOC_KY_INT']).reset_index(drop=True)
    
    # ---------------------------------------------------------
    # 2. XỬ LÝ SỐ LIỆU & LOGIC
    # ---------------------------------------------------------
    cols_float = ['GPA', 'CPA', 'DIEM_TRUNGTUYEN', 'DIEM_CHUAN']
    cols_int = ['TC_DANGKY', 'TC_HOANTHANH']
    
    for col in cols_float:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        
    for col in cols_int:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

    # Logic: Hoàn thành <= Đăng ký
    df['TC_HOANTHANH'] = np.minimum(df['TC_HOANTHANH'], df['TC_DANGKY'])
    
    # Target Transformation
    df['COMPLETION_RATE'] = df['TC_HOANTHANH'] / (df['TC_DANGKY'] + 1e-9)
    df['COMPLETION_RATE'] = df['COMPLETION_RATE'].clip(0, 1)

    # Clip điểm số
    df['GPA'] = df['GPA'].clip(0, 4.0)
    df['CPA'] = df['CPA'].clip(0, 4.0)

    # Admission Gap Feature
    if 'DIEM_TRUNGTUYEN' in df.columns and 'DIEM_CHUAN' in df.columns:
        df['ADMISSION_GAP'] = df['DIEM_TRUNGTUYEN'] - df['DIEM_CHUAN']
    
    # Lọc rác
    initial_len = len(df)
    df = df[df['TC_DANGKY'] > 0]
    
    print(f"--- ✅ HOÀN TẤT. Kích thước data: {df.shape} ---")
    print("Sample HOC_KY_INT:", df['HOC_KY_INT'].head().tolist())
    
    return df

# Chạy thử lại
df_clean = clean_data_pipeline_v3(admission, academic_records)
df_clean.head()

--- 🚀 BẮT ĐẦU QUY TRÌNH LÀM SẠCH DỮ LIỆU (FIXED VERSION) ---
-> Đang xử lý cột HOC_KY...
--- ✅ HOÀN TẤT. Kích thước data: (105726, 14) ---
Sample HOC_KY_INT: [20231, 20232, 20211, 20212, 20221]


,MA_SO_SV,HOC_KY,CPA,GPA,TC_DANGKY,TC_HOANTHANH,HOC_KY_INT,NAM_TUYENSINH,PTXT,TOHOP_XT,DIEM_TRUNGTUYEN,DIEM_CHUAN,COMPLETION_RATE,ADMISSION_GAP
0,00003e092652,HK1 2023-2024,1.64,1.97,18,15,20231,2023,100,A00,21.32,20.25,0.833333,1.07
1,00003e092652,HK2 2023-2024,1.53,2.05,18,13,20232,2023,100,A00,21.32,20.25,0.722222,1.07
2,000e15519006,HK1 2021-2022,3.85,3.85,9,9,20211,2021,1,D07,23.84,22.43,1.000000,1.41
3,000e15519006,HK2 2021-2022,2.77,3.12,19,19,20212,2021,1,D07,23.84,22.43,1.000000,1.41
4,000e15519006,HK1 2022-2023,2.83,2.98,21,21,20221,2021,1,D07,23.84,22.43,1.000000,1.41


In [4]:
import pandas as pd
import numpy as np
import re

# ==============================================================================
# 1. CÁC HÀM TIỆN ÍCH (UTILS)
# ==============================================================================

def parse_semester_string(sem_str):
    """
    Chuyển đổi 'HK1 2023-2024' -> 20231 (Int) để sort time-series.
    """
    s = str(sem_str).strip()
    if s.isdigit(): return int(s)
    
    digits = re.findall(r'\d+', s)
    if len(digits) >= 2:
        years = [int(d) for d in digits if len(d) == 4]
        sems = [int(d) for d in digits if len(d) == 1]
        if years and sems:
            return years[0] * 10 + sems[0]
    return 0

def fast_slope(y):
    """
    Tính hệ số góc (Trend) nhanh.
    Input: Array numpy (có thể chứa NaN).
    """
    # Lọc bỏ NaN trước khi tính (Quan trọng!)
    y_clean = y[~np.isnan(y)]
    n = len(y_clean)
    if n < 2: return 0.0
    
    x = np.arange(n)
    x_mean = np.mean(x)
    y_mean = np.mean(y_clean)
    
    numerator = np.sum((x - x_mean) * (y_clean - y_mean))
    denominator = np.sum((x - x_mean) ** 2)
    
    return numerator / (denominator + 1e-6)

# ==============================================================================
# 2. DATA PREPROCESSING PIPELINE
# ==============================================================================

def clean_data_pipeline(admission, academic_records, is_test=False):
    print("--- 🚀 1. PREPROCESSING DATA ---")
    adm = admission.copy()
    acad = academic_records.copy()
    
    # Chuẩn hóa ID
    adm['MA_SO_SV'] = adm['MA_SO_SV'].astype(str)
    acad['MA_SO_SV'] = acad['MA_SO_SV'].astype(str)
    
    # Tạo Time-Index
    acad['semester_order'] = acad['HOC_KY'].apply(parse_semester_string)
    
    # Merge
    df = pd.merge(acad, adm, on='MA_SO_SV', how='left')
    
    # Sort Time-Series (CỰC KỲ QUAN TRỌNG)
    df = df.sort_values(by=['MA_SO_SV', 'semester_order']).reset_index(drop=True)
    
    # Numeric conversion
    cols_float = ['GPA', 'CPA', 'DIEM_TRUNGTUYEN', 'DIEM_CHUAN']
    cols_int = ['TC_DANGKY', 'TC_HOANTHANH']
    
    for col in cols_float:
        if col in df.columns: df[col] = pd.to_numeric(df[col], errors='coerce')
    for col in cols_int:
        if col in df.columns: df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

    # Logic clean
    if not is_test:
        df['TC_HOANTHANH'] = np.minimum(df['TC_HOANTHANH'], df['TC_DANGKY'])
        df['GPA'] = df['GPA'].clip(0, 4.0)
        df['CPA'] = df['CPA'].clip(0, 4.0)
    
    # Target Transformation (Chỉ dùng cho train)
    if not is_test:
        df['COMPLETION_RATE'] = df['TC_HOANTHANH'] / (df['TC_DANGKY'] + 1e-9)
        df['COMPLETION_RATE'] = df['COMPLETION_RATE'].clip(0, 1)

    return df

# ==============================================================================
# 3. FEATURE ENGINEERING CLASS (CORE)
# ==============================================================================

class FeatureEngineer:    
    def __init__(self):
        # Các cột Category sẽ giữ lại
        self.cat_cols = ['PTXT', 'TOHOP_XT', 'MA_NGANH', 'KV_UT', 'KHOA_VIEN'] 

    def create_features(self, df):
        print("--- ⚡ 2. FEATURE ENGINEERING (FULL POWER) ---")
        df = df.copy()
        
        # Sort lại cho chắc chắn
        df = df.sort_values(['MA_SO_SV', 'semester_order']).reset_index(drop=True)
        
        # Convert Category
        for col in self.cat_cols:
            if col in df.columns:
                df[col] = df[col].astype(str).astype('category')

        # Groupby object
        g = df.groupby('MA_SO_SV')

        # ---------------------------------------------------------
        # A. BASE LAGS (Cực kỳ quan trọng: Dùng shift(1) để tránh Leakage)
        # ---------------------------------------------------------
        # Tạo biến Raw (chứa NaN) để tính toán thống kê chính xác hơn
        df['Prev_GPA_Raw'] = g['GPA'].shift(1)
        
        # Tạo biến Fill (để model dùng trực tiếp)
        df['Prev_GPA'] = df['Prev_GPA_Raw'].fillna(-1)
        df['Prev_CPA'] = g['CPA'].shift(1).fillna(-1)
        df['Prev_TC_HOANTHANH'] = g['TC_HOANTHANH'].shift(1).fillna(0)
        df['Prev_TC_DANGKY'] = g['TC_DANGKY'].shift(1).fillna(0)
        
        # Cờ năm nhất (Chưa có lịch sử)
        df['is_freshman'] = (df['Prev_TC_DANGKY'] == 0).astype(int)

        # ---------------------------------------------------------
        # B. GỌI CÁC NHÓM FEATURE
        # ---------------------------------------------------------
        df = self._create_admission_features(df)
        df = self._create_history_features(df)
        df = self._create_trend_features(df)  # <--- Slope, Volatility nằm ở đây
        df = self._create_risk_features(df)
        
        # Dọn dẹp cột tạm
        if 'Prev_GPA_Raw' in df.columns:
            df = df.drop(columns=['Prev_GPA_Raw'])
            
        return df

    def _create_admission_features(self, df):
        # Khoảng cách điểm đầu vào
        if 'DIEM_TRUNGTUYEN' in df.columns and 'DIEM_CHUAN' in df.columns:
            df['diem_vuot_chuan'] = df['DIEM_TRUNGTUYEN'] - df['DIEM_CHUAN']
        
        # Tuổi (giả định năm hiện tại 2025/2026)
        if 'NAM_TUYENSINH' in df.columns:
            df['nam_tuoi'] = 2026 - df['NAM_TUYENSINH']
            
        # Số kỳ đã học (tính từ năm nhập học)
        df['semester_number'] = df.groupby('MA_SO_SV').cumcount() + 1
        return df

    def _create_history_features(self, df):
        # 1. Delta GPA-CPA: Đang học tốt hơn hay tệ hơn trung bình tích lũy?
        df['prev_gpa_cpa_diff'] = df['Prev_GPA'] - df['Prev_CPA']
        
        # 2. Tỷ lệ hoàn thành kỳ trước
        df['prev_completion_rate'] = df['Prev_TC_HOANTHANH'] / (df['Prev_TC_DANGKY'] + 1e-9)
        
        # 3. Load Factor (Áp lực): Đăng ký kỳ này / Sức học trung bình
        avg_capacity = df.groupby('MA_SO_SV')['Prev_TC_HOANTHANH'].transform(
            lambda x: x.rolling(window=5, min_periods=1).mean()
        ).fillna(15)
        
        df['load_factor'] = df['TC_DANGKY'] / (avg_capacity + 1e-9)
        
        # Biến cờ: Rớt môn kỳ trước
        df['failed_last_sem'] = (df['Prev_TC_HOANTHANH'] < df['Prev_TC_DANGKY']).astype(int)
        
        return df

    def _create_trend_features(self, df):
        # Sử dụng Prev_GPA_Raw (có NaN) để tính Slope chính xác
        # Nếu dùng Prev_GPA (có -1), Slope sẽ bị sai lệch lớn
        g_raw = df.groupby('MA_SO_SV')['Prev_GPA_Raw']
        
        # 1. GPA Slope (Xu hướng điểm)
        df['gpa_trend_slope'] = g_raw.transform(
            lambda x: x.rolling(window=3, min_periods=2).apply(fast_slope, raw=True)
        ).fillna(0)
        
        # 2. [RECOVERED] GPA Volatility (Độ ổn định)
        df['gpa_volatility'] = g_raw.transform(
            lambda x: x.rolling(window=4, min_periods=2).std()
        ).fillna(0)
        
        # 3. Tích lũy tín chỉ (History)
        grouped = df.groupby('MA_SO_SV')
        cum_dangky = grouped['Prev_TC_DANGKY'].cumsum()
        cum_hoanthanh = grouped['Prev_TC_HOANTHANH'].cumsum()
        
        df['total_credits_failed'] = cum_dangky - cum_hoanthanh
        df['accumulated_fail_ratio'] = df['total_credits_failed'] / (cum_dangky + 1e-9)
        
        # 4. [RECOVERED] Credit Velocity (Tốc độ học)
        semester_count = grouped.cumcount() + 1
        df['credit_velocity'] = cum_hoanthanh / semester_count
        
        return df

    def _create_risk_features(self, df):
        # Hành vi "Gỡ gạc" (Aggressive Recovery): Rớt môn kỳ trước -> Đăng ký nhiều hơn kỳ này
        more_credits = (df['TC_DANGKY'] > df['Prev_TC_DANGKY'])
        df['aggressive_recovery'] = (df['failed_last_sem'] & more_credits).astype(int)
        
        # Kỳ vọng hoàn thành (Expected Credits) = Đăng ký * Tỷ lệ đậu toàn cục của SV đó
        df['expected_real_credits'] = df['TC_DANGKY'] * (1 - df['accumulated_fail_ratio'])
        
        return df

    def get_feature_columns(self, df):
        """Tự động lấy danh sách feature dựa trên prefix"""
        
        # Whitelist các prefix
        valid_prefixes = [
            'Prev_', 'prev_', 'sem_', 'diem_', 'nam_', 'is_', 
            'load_', 'aggressive_', 'gpa_', 'total_', 'accumulated_',
            'credit_', 'expected_', 'failed_'
        ]
        
        valid_exact = ['TC_DANGKY', 'DIEM_TRUNGTUYEN', 'DIEM_CHUAN', 'semester_number']
        valid_exact.extend(self.cat_cols)
        
        final_cols = []
        # Các cột target/meta cần loại bỏ
        ignore_cols = ['TC_HOANTHANH', 'GPA', 'CPA', 'semester_order', 'MA_SO_SV', 'HOC_KY', 'COMPLETION_RATE', 'Prev_GPA_Raw']
        
        for col in df.columns:
            if col in ignore_cols: continue
            
            is_valid = False
            if col in valid_exact: is_valid = True
            else:
                for prefix in valid_prefixes:
                    if col.startswith(prefix):
                        is_valid = True
                        break
            
            if is_valid: final_cols.append(col)
                
        return final_cols

# ==============================================================================
# 4. EXECUTION BLOCK (CHẠY THỬ)
# ==============================================================================


test_raw = pd.read_csv(TEST_PATH)

print("--- BẮT ĐẦU PIPELINE ---")

# 1. Clean Data
df_train_raw = clean_data_pipeline(admission, academic_records, is_test=False)

# 2. Chuẩn bị Test (Gán dummy values để chạy được pipeline)
test_copy = test_raw.copy()
for col in ['TC_HOANTHANH', 'GPA', 'CPA']: test_copy[col] = 0
df_test_raw = clean_data_pipeline(admission, test_copy, is_test=True)

# 3. Gộp Train + Test để tạo feature (Tránh biên giới giữa các kỳ bị đứt gãy)
df_train_raw['set_type'] = 'TRAIN'
df_test_raw['set_type'] = 'TEST'
full_df = pd.concat([df_train_raw, df_test_raw], ignore_index=True)

# 4. Feature Engineering
fe = FeatureEngineer()
full_df_fe = fe.create_features(full_df)

# 5. Tách lại Train/Test
train_final = full_df_fe[full_df_fe['set_type'] == 'TRAIN'].copy()
test_final = full_df_fe[full_df_fe['set_type'] == 'TEST'].copy()

# 6. Lấy features & Target
feature_cols = fe.get_feature_columns(train_final)
X = train_final[feature_cols]
y = train_final['TC_HOANTHANH'] # Hoặc dùng COMPLETION_RATE nếu muốn

print(f"✅ Xong! Số lượng Features: {len(feature_cols)}")
print(f"Sample Features: {feature_cols[:5]}")

--- BẮT ĐẦU PIPELINE ---
--- 🚀 1. PREPROCESSING DATA ---
--- 🚀 1. PREPROCESSING DATA ---
--- ⚡ 2. FEATURE ENGINEERING (FULL POWER) ---
✅ Xong! Số lượng Features: 24
Sample Features: ['TC_DANGKY', 'PTXT', 'TOHOP_XT', 'DIEM_TRUNGTUYEN', 'DIEM_CHUAN']


In [5]:
import optuna
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

print("--- 🚀 TRAINING PHASE (CATBOOST) ---")

SPLIT_SEM = 20231
VALID_SEM = 20232

df_modeling = full_df_fe[full_df_fe['set_type'] == 'TRAIN'].copy()
train_mask = df_modeling['semester_order'] <= SPLIT_SEM
valid_mask = df_modeling['semester_order'] == VALID_SEM

X_train = df_modeling[train_mask][feature_cols].copy()
y_train = df_modeling[train_mask]['COMPLETION_RATE']
w_train = df_modeling[train_mask]['TC_DANGKY']

X_valid = df_modeling[valid_mask][feature_cols].copy()
y_valid_rate = df_modeling[valid_mask]['COMPLETION_RATE']
y_valid_credits = df_modeling[valid_mask]['TC_HOANTHANH']
valid_credits_dangky = df_modeling[valid_mask]['TC_DANGKY']

categorical_features = X_train.select_dtypes(include=['object', 'category']).columns.tolist()

print(f"Train: {X_train.shape} | Valid: {X_valid.shape}")
if categorical_features:
    print(f"Categorical features: {len(categorical_features)} detected")

def objective(trial):
    params = {
        'iterations': 2000,
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.05),
        'depth': trial.suggest_int('depth', 3, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'random_strength': trial.suggest_float('random_strength', 0.5, 5.0),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 20),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'rsm': trial.suggest_float('rsm', 0.6, 1.0),
        'bootstrap_type': 'Bernoulli',  # ← Fix: Bernoulli để dùng được subsample
        'loss_function': 'RMSE',
        'random_seed': 42,
        'thread_count': -1,
        'verbose': False,
        'cat_features': categorical_features if categorical_features else None
    }
    
    model = CatBoostRegressor(**params)
    model.fit(
        X_train, y_train,
        eval_set=(X_valid, y_valid_rate),
        early_stopping_rounds=100,
        sample_weight=w_train.values if w_train is not None else None,
        verbose=False
    )
    
    pred_rate = model.predict(X_valid)
    pred_credits = np.minimum(pred_rate * valid_credits_dangky, valid_credits_dangky)
    return np.sqrt(mean_squared_error(y_valid_credits, pred_credits))

# ✅ CHẠY OPTUNA VỚI LOG MẶC ĐỊNH (KHÔNG THÊM GÌ CẢ)
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)  # ← Không có callbacks, không có show_progress_bar

print("\n--- ✅ OPTUNA FINISHED ---")
print(f"Best RMSE: {study.best_value:.4f}")
print("Best Params:", study.best_params)

# Train final model
best_params = study.best_params.copy()
best_params.update({
    'iterations': 2000,
    'loss_function': 'RMSE',
    'random_seed': 42,
    'thread_count': -1,
    'verbose': False,
    'bootstrap_type': 'Bernoulli',
    'cat_features': categorical_features if categorical_features else None
})

model_cb = CatBoostRegressor(**best_params)
model_cb.fit(
    X_train, y_train,
    eval_set=(X_valid, y_valid_rate),
    early_stopping_rounds=100,
    sample_weight=w_train.values if w_train is not None else None,
    verbose=False
)
print(f"\n✅ Final model trained ({model_cb.tree_count_} trees)")

[I 2026-02-04 01:20:15,050] A new study created in memory with name: no-name-68204a14-9b79-438b-919c-d569773c05bf


--- 🚀 TRAINING PHASE (CATBOOST) ---
Train: (90582, 24) | Valid: (15144, 24)
Categorical features: 2 detected


[I 2026-02-04 01:20:54,949] Trial 0 finished with value: 3.7647938543557884 and parameters: {'learning_rate': 0.020125122806509764, 'depth': 4, 'l2_leaf_reg': 6.394669210571399, 'random_strength': 2.5505972389300653, 'min_data_in_leaf': 6, 'subsample': 0.9129544836328539, 'rsm': 0.7865221705115406}. Best is trial 0 with value: 3.7647938543557884.
[I 2026-02-04 01:21:45,812] Trial 1 finished with value: 3.765662893969169 and parameters: {'learning_rate': 0.02204424511294725, 'depth': 5, 'l2_leaf_reg': 1.132572610439031, 'random_strength': 4.28875744982133, 'min_data_in_leaf': 11, 'subsample': 0.630565860654738, 'rsm': 0.6414295000207022}. Best is trial 0 with value: 3.7647938543557884.
[I 2026-02-04 01:22:52,206] Trial 2 finished with value: 3.779204436575518 and parameters: {'learning_rate': 0.01057782897556377, 'depth': 3, 'l2_leaf_reg': 4.571478494211705, 'random_strength': 4.283802833529106, 'min_data_in_leaf': 15, 'subsample': 0.9210142825145325, 'rsm': 0.7080611878117805}. Best is


--- ✅ OPTUNA FINISHED ---
Best RMSE: 3.7375
Best Params: {'learning_rate': 0.0256259812174494, 'depth': 8, 'l2_leaf_reg': 7.717992409061578, 'random_strength': 0.6167134246647357, 'min_data_in_leaf': 2, 'subsample': 0.7968123245621603, 'rsm': 0.8573055037828949}

✅ Final model trained (403 trees)


In [10]:
y_pred_rate = model_cb.predict(X_valid)
y_pred = y_pred_rate * valid_credits_dangky

In [11]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

# === TRY IMPORT OPTIONAL DEPENDENCIES ===
SHAP_AVAILABLE = False
LIME_AVAILABLE = False

try:
    import shap
    SHAP_AVAILABLE = True
except Exception as e:
    print(f"⚠️ SHAP not available: {str(e)[:80]}")

try:
    import lime
    import lime.lime_tabular
    LIME_AVAILABLE = True
except Exception as e:
    print(f"⚠️ LIME not available: {str(e)[:80]}")

# ========================================
# METRICS (LUÔN HOẠT ĐỘNG)
# ========================================

def calculate_mape(y_true, y_pred, epsilon=1e-8):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    mask = np.abs(y_true) > epsilon
    if not np.any(mask):
        return np.nan
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

def calculate_regression_metrics(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    mape = calculate_mape(y_true, y_pred)
    return {'rmse': rmse, 'r2': r2, 'mape': mape}

def print_metrics(metrics, prefix=""):
    print(f"{prefix}RMSE: {metrics['rmse']:.4f} | R²: {metrics['r2']:.4f} | MAPE: {metrics['mape']:.2f}%")

# ========================================
# FEATURE IMPORTANCE (LUÔN HOẠT ĐỘNG)
# ========================================

def get_feature_importance(model, feature_names=None):
    if hasattr(model, 'feature_importances_'):
        importance = model.feature_importances_
    elif hasattr(model, 'get_feature_importance'):  # CatBoost
        importance = model.get_feature_importance()
    else:
        raise ValueError("Model không hỗ trợ feature importance")
    
    if feature_names is None:
        feature_names = [f"Feature_{i}" for i in range(len(importance))]
    
    df = pd.DataFrame({
        'feature': feature_names,
        'importance': importance
    }).sort_values('importance', ascending=False).reset_index(drop=True)
    df['importance_pct'] = (df['importance'] / df['importance'].sum() * 100).round(2)
    return df

def plot_feature_importance(importance_df, top_n=20, title="Feature Importance"):
    df_plot = importance_df.head(top_n).copy()
    df_plot['feature'] = df_plot['feature'].astype(str)
    
    fig = px.bar(
        df_plot, x='importance', y='feature', orientation='h',
        color='importance', color_continuous_scale='Viridis',
        text=df_plot['importance_pct'].apply(lambda x: f"{x}%"),
        title=title
    )
    fig.update_layout(
        yaxis=dict(autorange="reversed"),
        xaxis_title="Importance",
        yaxis_title="Feature",
        height=max(400, top_n * 25),
        template="plotly_white"
    )
    fig.update_traces(textposition='outside')
    return fig

# ========================================
# DATA PREPROCESSING FOR EXPLAINABILITY
# ========================================

def prepare_data_for_explainers(X, fit_encoders=False, encoders=None):
    """
    Chuyển đổi categorical features → numeric cho SHAP/LIME.
    Trả về: (X_numeric, feature_names, categorical_info, encoders)
    """
    if isinstance(X, pd.DataFrame):
        X = X.copy()
    else:
        X = pd.DataFrame(X)
    
    categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
    numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
    
    new_encoders = {} if encoders is None else encoders.copy()
    X_processed = X.copy()
    
    if categorical_cols:
        if fit_encoders:
            # Fit mới encoders
            for col in categorical_cols:
                le = LabelEncoder()
                # Xử lý missing values trước khi encode
                X_processed[col] = X_processed[col].fillna('MISSING').astype(str)
                le.fit(X_processed[col])
                X_processed[col] = le.transform(X_processed[col])
                new_encoders[col] = le
        else:
            # Dùng encoders có sẵn
            for col in categorical_cols:
                if col in new_encoders:
                    X_processed[col] = X_processed[col].fillna('MISSING').astype(str)
                    X_processed[col] = new_encoders[col].transform(X_processed[col])
                else:
                    # Fallback: gán giá trị mặc định
                    X_processed[col] = 0
                    warnings.warn(f"⚠️ Categorical encoder missing for '{col}', using default value 0")
    
    # Đảm bảo tất cả cột là numeric
    X_processed = X_processed.astype(float)
    
    return (
        X_processed.values,
        X.columns.tolist(),
        {'categorical_cols': categorical_cols, 'numeric_cols': numeric_cols},
        new_encoders
    )

# ========================================
# SHAP (XỬ LÝ CATEGORICAL TỰ ĐỘNG)
# ========================================

def get_shap_values(model, X_sample, model_type=None):
    if not SHAP_AVAILABLE:
        raise RuntimeError("SHAP chưa được cài đặt")
    
    # Chuẩn bị dữ liệu numeric
    X_num, feature_names, _, _ = prepare_data_for_explainers(X_sample, fit_encoders=False)
    
    if model_type == 'catboost':
        explainer = shap.TreeExplainer(model, feature_perturbation="tree_path_dependent")
    else:
        explainer = shap.TreeExplainer(model)
    
    shap_values = explainer.shap_values(X_num)
    return shap_values, feature_names, X_num

def plot_shap_summary(shap_values, X_sample, feature_names, max_display=20, title="SHAP Summary Plot"):
    if not SHAP_AVAILABLE:
        print("⚠️ SHAP không khả dụng - skip plot")
        return None
    
    # Tạo DataFrame SHAP
    shap_df = pd.DataFrame(shap_values, columns=feature_names)
    mean_abs_shap = shap_df.abs().mean().sort_values(ascending=False)
    top_features = mean_abs_shap.head(max_display).index.tolist()
    
    # Chuẩn bị dữ liệu cho beeswarm-style plot
    plot_data = []
    for feature in top_features[::-1]:  # reverse để feature quan trọng nhất ở trên
        shap_vals = shap_df[feature].values
        feat_vals = X_sample[:, feature_names.index(feature)]
        colors = feat_vals  # dùng giá trị feature để tô màu
        
        for i in range(len(shap_vals)):
            plot_data.append({
                'feature': feature,
                'shap_value': shap_vals[i],
                'feature_value': colors[i]
            })
    
    df_plot = pd.DataFrame(plot_data)
    
    # Tạo beeswarm plot bằng Plotly
    fig = px.scatter(
        df_plot,
        x='shap_value',
        y='feature',
        color='feature_value',
        color_continuous_scale='RdBu',
        title=title,
        height=max(400, len(top_features) * 30),
        hover_data=['feature_value']
    )
    
    fig.update_layout(
        xaxis_title="SHAP Value (impact on model output)",
        yaxis_title="Feature",
        template="plotly_white",
        showlegend=True
    )
    fig.add_vline(x=0, line_dash="dash", line_color="gray", line_width=1)
    return fig

# ========================================
# LIME (XỬ LÝ CATEGORICAL ĐÚNG CÁCH)
# ========================================

def explain_instance_lime(model, X_train, X_instance, feature_names=None, 
                         categorical_features=None, categorical_names=None, 
                         num_features=10, num_samples=5000):
    if not LIME_AVAILABLE:
        raise RuntimeError("LIME chưa được cài đặt")
    
    # Chuẩn bị training data cho LIME (numeric)
    X_train_num, train_feat_names, cat_info, encoders = prepare_data_for_explainers(
        X_train, fit_encoders=True
    )
    
    # Chuẩn bị instance cần giải thích
    if isinstance(X_instance, pd.DataFrame):
        X_instance = X_instance.copy()
    else:
        X_instance = pd.DataFrame(X_instance, columns=train_feat_names)
    
    X_instance_num, _, _, _ = prepare_data_for_explainers(
        X_instance, fit_encoders=False, encoders=encoders
    )
    
    # Xác định categorical feature indices
    if categorical_features is None:
        categorical_features = [
            i for i, col in enumerate(train_feat_names) 
            if col in cat_info['categorical_cols']
        ]
    
    # Tạo categorical_names mapping
    if categorical_names is None and cat_info['categorical_cols']:
        categorical_names = {}
        for col in cat_info['categorical_cols']:
            idx = train_feat_names.index(col)
            le = encoders.get(col)
            if le:
                categorical_names[idx] = list(le.classes_)
    
    # Tạo explainer với categorical info
    explainer = lime.lime_tabular.LimeTabularExplainer(
        X_train_num,
        feature_names=train_feat_names,
        categorical_features=categorical_features,
        categorical_names=categorical_names,
        mode='regression',
        verbose=False,
        random_state=42,
        kernel_width=3,
        discretize_continuous=False  # Giữ nguyên giá trị liên tục
    )
    
    # Giải thích instance
    exp = explainer.explain_instance(
        X_instance_num[0],
        model.predict,
        num_features=num_features,
        num_samples=num_samples
    )
    
    return exp, encoders

def plot_lime_explanation(lime_explanation, title="LIME Explanation"):
    if not LIME_AVAILABLE:
        print("⚠️ LIME không khả dụng - skip plot")
        return None
    
    exp_list = lime_explanation.as_list()
    df = pd.DataFrame(exp_list, columns=['feature', 'weight'])
    df = df.sort_values('weight', key=abs, ascending=True).reset_index(drop=True)
    colors = ['green' if w > 0 else 'red' for w in df['weight']]
    
    fig = go.Figure(go.Bar(
        y=df['feature'], 
        x=df['weight'], 
        orientation='h',
        marker_color=colors,
        text=df['weight'].apply(lambda x: f"{x:+.3f}"),
        textposition='auto',
    ))
    
    fig.update_layout(
        title=title,
        xaxis_title="Feature Contribution to Prediction",
        yaxis_title="Feature",
        template="plotly_white",
        height=300 + len(df) * 35,
        xaxis=dict(showgrid=True, zeroline=True, zerolinewidth=2, zerolinecolor='gray')
    )
    fig.add_vline(x=0, line_dash="dash", line_color="gray", line_width=1)
    return fig

# ========================================
# HELPER: SHOW ALL METRICS & PLOTS
# ========================================

def show_all(model, X_valid, y_valid, y_pred, 
             X_train=None, model_type=None, 
             instance_idx=0, top_n_features=15):
    """
    Hiển thị đầy đủ metrics + visualizations cho model.
    
    Parameters:
    -----------
    model : model đã train (XGBoost/LightGBM/CatBoost)
    X_valid : pd.DataFrame - validation features
    y_valid : array-like - true labels (credits)
    y_pred : array-like - predicted credits
    X_train : pd.DataFrame - training features (cần cho LIME)
    model_type : str - 'xgboost', 'lightgbm', 'catboost'
    instance_idx : int - index của instance để giải thích bằng SHAP/LIME
    top_n_features : int - số feature hiển thị trong plots
    """
    # 1. Metrics
    print("="*60)
    print("📊 REGRESSION METRICS")
    print("="*60)
    metrics = calculate_regression_metrics(y_valid, y_pred)
    print_metrics(metrics, "Validation: ")
    print()
    
    # 2. Feature Importance
    print("="*60)
    print("📈 FEATURE IMPORTANCE")
    print("="*60)
    imp_df = get_feature_importance(model, feature_names=X_valid.columns.tolist())
    fig_imp = plot_feature_importance(imp_df, top_n=top_n_features)
    fig_imp.show()
    print()
    
    # 3. SHAP Summary (nếu có)
    if SHAP_AVAILABLE and X_valid.shape[0] > 0:
        print("="*60)
        print("🔍 SHAP SUMMARY PLOT")
        print("="*60)
        try:
            # Sample 100 instances để tính SHAP nhanh hơn
            sample_size = min(100, len(X_valid))
            X_sample = X_valid.iloc[:sample_size] if isinstance(X_valid, pd.DataFrame) else X_valid[:sample_size]
            
            shap_vals, feat_names, X_num = get_shap_values(model, X_sample, model_type=model_type)
            fig_shap = plot_shap_summary(shap_vals, X_num, feat_names, max_display=top_n_features)
            if fig_shap:
                fig_shap.show()
        except Exception as e:
            print(f"⚠️ SHAP error: {str(e)[:100]}")
    print()
    
    # 4. LIME Explanation (nếu có)
    if LIME_AVAILABLE and X_train is not None and len(X_valid) > instance_idx:
        print("="*60)
        print(f"🔬 LIME EXPLANATION (Instance #{instance_idx})")
        print("="*60)
        try:
            instance = X_valid.iloc[[instance_idx]] if isinstance(X_valid, pd.DataFrame) else X_valid[instance_idx:instance_idx+1]
            lime_exp, _ = explain_instance_lime(
                model, 
                X_train=X_train, 
                X_instance=instance,
                feature_names=X_valid.columns.tolist(),
                num_features=top_n_features
            )
            fig_lime = plot_lime_explanation(lime_exp, title=f"LIME: Instance #{instance_idx}")
            if fig_lime:
                fig_lime.show()
        except Exception as e:
            print(f"⚠️ LIME error: {str(e)[:100]}")
    print()
    print("="*60)
    print("✅ DONE")
    print("="*60)

In [12]:
show_all(
    model=model_cb,
    X_valid=X_valid,
    y_valid=y_valid_credits,
    y_pred=y_pred,
    X_train=X_train,          # Cần cho LIME
    model_type='lightgbm',    # 'xgboost', 'lightgbm', 'catboost'
    instance_idx=0,           # Instance nào để giải thích
    top_n_features=15
)

📊 REGRESSION METRICS
Validation: RMSE: 3.7402 | R²: 0.7162 | MAPE: 29.85%

📈 FEATURE IMPORTANCE



🔍 SHAP SUMMARY PLOT
⚠️ SHAP error: 'data' is numpy array of floating point numerical type, it means no categorical features, but 'cat_f

🔬 LIME EXPLANATION (Instance #0)
⚠️ LIME error: 'data' is numpy array of floating point numerical type, it means no categorical features, but 'cat_f

✅ DONE


In [13]:
def submission(model, output_filename):
    print("\n--- 4. PREDICTING TEST SET (HK1 2024-2025) ---")
    test_final = full_df_fe[full_df_fe['set_type'] == 'TEST'].copy()
    X_test = test_final[feature_cols]
    test_dangky = test_final['TC_DANGKY']

    # Dự báo
    pred_rate_test = model.predict(X_test)

    # Hậu xử lý tương tự
    pred_rate_test = np.clip(pred_rate_test, 0, 1)
    pred_credits_test = pred_rate_test * test_dangky
    pred_credits_test = np.minimum(pred_credits_test, test_dangky)

    final_submission = pred_credits_test

    # Tạo DataFrame nộp bài
    submission_df = pd.DataFrame({
        'MA_SO_SV': test_final['MA_SO_SV'],
        'PRED_TC_HOANTHANH': final_submission
    })

    # Lưu file
    submission_df.to_csv(output_filename, index=False)
    print(f"✅ Đã lưu file kết quả: {output_filename}")
    print(submission_df.head())

In [14]:
submission(model=model_cb, output_filename="submission_catboost_v1.csv")


--- 4. PREDICTING TEST SET (HK1 2024-2025) ---
✅ Đã lưu file kết quả: submission_catboost_v1.csv
        MA_SO_SV  PRED_TC_HOANTHANH
2   00003e092652          15.368979
3   00027b0dec4c          17.220308
10  000e15519006          17.561364
13  000ea6e12003          16.335211
16  00109b845a3d           4.893423
